In [1]:
from azureml.core import Workspace, Experiment


ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-124322
Azure region: southcentralus
Subscription id: e8f628b3-bb5b-4edf-947a-8637ca6ea7c2
Resource group: aml-quickstarts-124322


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
import pandas as pd

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = exp.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpu-cluster-4"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import shutil
import os

# Specify parameter sampler
from azureml.train.hyperdrive import choice, loguniform
ps = RandomParameterSampling(
    {
        '--C': choice(0.2, 0,5, 1),
        '--max_iter': choice(25, 50, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
shutil.copy('train.py', "./training")
est = SKLearn(
    source_directory= "./training",
    compute_target=compute_target,
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3f324f96-eeef-4975-8a0f-7f303c91f745
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_3f324f96-eeef-4975-8a0f-7f303c91f745?wsid=/subscriptions/e8f628b3-bb5b-4edf-947a-8637ca6ea7c2/resourcegroups/aml-quickstarts-124322/workspaces/quick-starts-ws-124322

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-03T03:08:06.414332][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2020-11-03T03:08:06.231255][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-03T03:08:05.635226][API][INFO]Experiment created<END>\n"<START>[2020-11-03T03:08:08.3104128Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_3f324f96-eeef-4975-8a0f-7f303c91f745
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_3f324f96-eeef-4975-8a0f-7f303c91f745?wsid=/subscriptions/e8f62

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())


['--C', '0.2', '--max_iter', '25']
['azureml-logs/55_azureml-execution-tvmps_d749d9cdb01add5e1690115fb6dbfbf9bb8df42f1f9ce39c5dd55ca8eb9681ab_d.txt', 'azureml-logs/65_job_prep-tvmps_d749d9cdb01add5e1690115fb6dbfbf9bb8df42f1f9ce39c5dd55ca8eb9681ab_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_d749d9cdb01add5e1690115fb6dbfbf9bb8df42f1f9ce39c5dd55ca8eb9681ab_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [6]:
model = best_run.register_model(model_name='first', model_path='outputs/model.joblib')

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data)


In [8]:
import train
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
scaled_data = scaler.transform(x)
scaled_data = pd.DataFrame(scaled_data, columns= x.columns)
x_train, x_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.2, random_state=1)
x_df = x_train.assign(y_train=y_train.values)
df = pd.DataFrame(x_df)
df.to_csv("training/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./training', target_path='udacity-project', overwrite=True, show_progress=True)
from azureml.core import Dataset
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('udacity-project/train_data.csv'))


Uploading an estimated of 2 files
Uploading ./training/train.py
Uploaded ./training/train.py, 1 files out of an estimated total of 2
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
}

automl_config = AutoMLConfig(task = 'classification',
                             training_data = train_data,
                             label_column_name = "y_train",
                             **automl_settings
                            )


In [10]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()


WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


{'runId': 'AutoML_114e2ec9-1411-42c0-9d8a-f462a7414059',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-03T03:20:15.005072Z',
 'endTimeUtc': '2020-11-03T03:52:32.891964Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"e8f628b3-bb5b-4edf-947a-8637ca6ea7c2","resource_group":"aml-quickstarts-124322","workspace_name":"quick-starts-ws-124322","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min"

In [11]:
# Retrieve and save your best automl model.

best_model_auto, fitted_model = remote_run.get_output()


In [13]:
best_run_metrics = best_model_auto.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

balanced_accuracy 0.7622394680262075
weighted_accuracy 0.9552541773635637
precision_score_weighted 0.9123435920357856
recall_score_weighted 0.9167678300455235
average_precision_score_macro 0.820813417463534
recall_score_macro 0.7622394680262075
matthews_correlation 0.5586510553487425
log_loss 0.18866726637130202
average_precision_score_weighted 0.9541886279754457
f1_score_weighted 0.9141480429765787
f1_score_micro 0.9167678300455235
recall_score_micro 0.9167678300455235
f1_score_macro 0.7782995984857746
precision_score_micro 0.9167678300455235
norm_macro_recall 0.5244789360524148
precision_score_macro 0.7975331181387502
AUC_macro 0.9456352125242931
AUC_micro 0.98014823927365
average_precision_score_micro 0.9809255859363812
accuracy 0.9167678300455235
AUC_weighted 0.9456352125242932
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_114e2ec9-1411-42c0-9d8a-f462a7414059_24/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_114e2ec9-1411-42c0-9d8a-f462a741

In [14]:
best_model_auto.register_model(model_path='outputs/model.pkl', model_name='automl', tags={'Training context':'Auto ML'}, properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

Model(workspace=Workspace.create(name='quick-starts-ws-124322', subscription_id='e8f628b3-bb5b-4edf-947a-8637ca6ea7c2', resource_group='aml-quickstarts-124322'), name=automl, id=automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC': '0.9456352125242932', 'Accuracy': '0.9167678300455235'})

In [15]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

